In [1]:
import pandas as pd
import numpy as np
import csv

In [2]:
# Data pre-processing And Feature Sellection

# Read data
covidData = pd.read_csv("./data/owid-covid-data.csv")
# Drop not wanted columns
covidData.drop(["continent","iso_code","human_development_index","new_deaths_smoothed","total_cases_per_million","new_cases_per_million","new_cases_smoothed_per_million","gdp_per_capita","extreme_poverty","cardiovasc_death_rate","diabetes_prevalence","female_smokers","male_smokers","handwashing_facilities","life_expectancy"],axis=1,inplace=True)
covidData.drop(["total_cases","new_cases_smoothed","total_deaths_per_million","new_deaths_per_million","new_deaths_smoothed_per_million","reproduction_rate","new_tests_smoothed","new_tests_smoothed_per_thousand","positive_rate","tests_per_case","stringency_index","population","population_density","median_age","aged_65_older"],axis=1,inplace=True)
covidData.drop(["total_deaths","icu_patients","icu_patients_per_million","hosp_patients","hosp_patients_per_million","weekly_icu_admissions","weekly_icu_admissions_per_million","weekly_hosp_admissions","weekly_hosp_admissions_per_million","total_tests","new_tests","total_tests_per_thousand","new_tests_per_thousand","tests_units"],axis=1,inplace=True)

covidData = pd.DataFrame(covidData)
covidData = covidData.fillna(0)
# Dropping row 'Wrold', considered to be of no use 
covidData.drop(covidData[covidData['location'] == 'World'].index,inplace=True)
# Dropping row 'Hong Kong' due to data absence of data
covidData.drop(covidData[covidData['location'] == 'Hong Kong'].index,inplace=True)

trainOlder70 = []
trainMonths = []
trainCases = {}
trainDeaths = []
trainHospitalBedsPer_k = []

testOlder70 = []
testMonths = []
testCases = {}
testDeaths = []
testHospitalBedsPer_k = []

# Gather features and store them in dataframes
targetCountries = ['Greece','Cuba','United Kingdom','Vietnam']
for index,row in covidData.iterrows():
    if int(row["date"].split("-")[0]) < 2021:
        if row["location"] not in trainCases:
            trainDeaths.append(row['new_deaths'])
            trainOlder70.append(row["aged_70_older"])
            trainHospitalBedsPer_k.append(row["hospital_beds_per_thousand"])
            trainCases[row["location"]] = {int(row["date"].split("-")[1]): int(row['new_cases'])}
            trainCases[row["location"]] = {13: row['new_cases']}
        else:
            if int(row["date"].split("-")[1]) in trainCases[row["location"]]:
                trainCases[row["location"]][int(row["date"].split("-")[1])] += int(row['new_cases'])
            else:
                trainCases[row["location"]][int(row["date"].split("-")[1])] = int(row['new_cases'])
            trainCases[row["location"]][13] += int(row['new_cases'])
            trainDeaths[-1] += row['new_deaths']
    else:
        print(row["date"])
        if row["location"] in targetCountries:
            if row["location"] not in testCases:
                testDeaths.append(row['new_deaths'])
                testOlder70.append(row["aged_70_older"])
                testHospitalBedsPer_k.append(row["hospital_beds_per_thousand"])
                testCases[row["location"]] = {int(row["date"].split("-")[1]): int(row['new_cases'])}
                testCases[row["location"]] = {13: row['new_cases']}
            else:
                if int(row["date"].split("-")[1]) in trainCases[row["location"]]:
                    testCases[row["location"]][int(row["date"].split("-")[1])] += int(row['new_cases'])
                else:
                    testCases[row["location"]][int(row["date"].split("-")[1])] = int(row['new_cases'])
                testCases[row["location"]][13] += int(row['new_cases'])
                testDeaths[-1] += row['new_deaths']

# Train dataframe
trainCovidData = pd.DataFrame()
trainCovidData["country"] = list(trainCases.keys())
# Gathering the number of monthly cases for each country
for i in range(13):
    monthlyCases = []
    for country,months in trainCases.items():
        for month,cases in months.items():
            if month == i + 1:
                monthlyCases.append(cases)
                break
    if i + 1 != 13:
        trainCovidData["casesforMonth-"+str(i+1)] = monthlyCases
    else:
        trainCovidData["totalCases"] = monthlyCases
    monthlyCases.clear()
trainCovidData["deaths"] = trainDeaths
trainCovidData["older70"] = trainOlder70
trainCovidData[u"beds‰"] = trainHospitalBedsPer_k


# Test dataframe
testCovidData = pd.DataFrame()
testCovidData["country"] = list(testCases.keys())
for i in range(13):
    monthlyCases = []
    for country,months in testCases.items():
        for month,cases in months.items():
            if month == i + 1:
                monthlyCases.append(cases)
                break
    if i + 1 != 13:
        testCovidData["casesforMonth-"+str(i+1)] = monthlyCases
    else:
        testCovidData["totalCases"] = monthlyCases
    monthlyCases.clear()
testCovidData["deaths"] = testDeaths
testCovidData["older70"] = testOlder70
testCovidData[u"beds‰"] = testHospitalBedsPer_k


display(trainCovidData)
display(testCovidData)

,country,casesforMonth-1,casesforMonth-2,casesforMonth-3,casesforMonth-4,casesforMonth-5,casesforMonth-6,casesforMonth-7,casesforMonth-8,casesforMonth-9,casesforMonth-10,casesforMonth-11,casesforMonth-12,totalCases,deaths,older70,beds‰
0,Afghanistan,0,1,174,1952,13081,16299,5158,1494,1109,2157,4849,1032,47306.0,1874.0,1.337,0.50
1,Albania,0,0,243,530,364,1398,2741,4237,4136,7226,17307,4806,42988.0,905.0,8.643,2.89
2,Algeria,0,1,715,3290,5388,4513,16487,14100,7036,6412,25257,5053,88252.0,2516.0,3.857,1.90
3,Andorra,0,0,376,369,19,91,70,251,874,2706,1989,305,7050.0,78.0,0.000,0.00
4,Angola,0,0,7,20,59,198,864,1506,2318,5833,4334,452,15591.0,354.0,1.362,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185,Vietnam,0,14,196,58,58,27,203,486,50,86,167,19,1366.0,35.0,4.718,2.60
186,Western Sahara,0,0,0,6,3,1,0,0,0,0,0,0,10.0,1.0,1.380,0.00
187,Yemen,0,0,0,6,317,835,570,230,76,29,128,146,2337.0,639.0,1.583,0.70
188,Zambia,0,0,35,71,951,537,4369,6134,2662,1673,1215,269,17916.0,364.0,1.542,2.00


,country,casesforMonth-1,casesforMonth-2,casesforMonth-3,casesforMonth-4,casesforMonth-5,casesforMonth-6,casesforMonth-7,casesforMonth-8,casesforMonth-9,casesforMonth-10,casesforMonth-11,casesforMonth-12,totalCases,deaths,older70,beds‰
